## PyNWB Tutorial
<br> Most of this comes from: https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html#sphx-glr-tutorials-domain-ecephys-py
<br> Also check out the PyNWBtutorial_ecephys.ipynb notebook

In [1]:
import os
import sys
import time
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np
import pandas as pd
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.behavior import BehavioralTimeSeries
from pynwb.ecephys import ElectricalSeries, LFP
from pynwb.misc import Units

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

## Specify file location

In [4]:
NWB_output_dir = r"E:\NWB_testing\mouse000000"

## Creating the NWB file

In [5]:
nwbfile = NWBFile(
    session_description="my first test recording",
    identifier="000000_day1",
    session_start_time=datetime.now(tzlocal()),
#     experimenter="Claar, Leslie D", # optional
#     lab="Bag End Laboratory", # optional
    institution="Allen Institute", # optional
    experiment_description="test making NWB file", # optional
    session_id="exp1", # optional
    related_publications="https://doi.org/10.7554/eLife.84630.1", # optional
)

Other optional fields may exist. See PyNWB docs.

## Add subject info

In the :py:class:`~pynwb.file.Subject` object we can store information about the experimental subject,
such as ``age``, ``species``, ``genotype``, ``sex``, and a ``description``.

The fields in the :py:class:`~pynwb.file.Subject` object are all free-form text (any format will be valid),
however it is recommended to follow particular conventions to help software tools interpret the data:

* **age**: [ISO 8601 Duration format](https://en.wikipedia.org/wiki/ISO_8601#Durations), e.g., ``"P90D"`` for 90 days old
* **species**: The formal latin binomial nomenclature, e.g., ``"Mus musculus"``, ``"Homo sapiens"``
* **sex**: Single letter abbreviation, e.g., ``"F"`` (female), ``"M"`` (male), ``"U"`` (unknown), and ``"O"`` (other)

Add the subject information to the :py:class:`~pynwb.file.NWBFile`
by setting the ``subject`` field to the new :py:class:`~pynwb.file.Subject` object.

In [6]:
nwbfile.subject = Subject(
    subject_id = "001",
    age = "P90D", # optional
    description = "mouse000000", # optional
    species = "Mus musculus", # optional
    sex = "M", # optional
    strain = "C57BL/6J", # optional
)

Can also include "genotype"

In [7]:
print(nwbfile)

root pynwb.file.NWBFile at 0x2239205778184
Fields:
  experiment_description: test making NWB file
  file_create_date: [datetime.datetime(2023, 3, 8, 12, 6, 36, 468983, tzinfo=tzlocal())]
  identifier: 000000_day1
  institution: Allen Institute
  related_publications: ['https://doi.org/10.7554/eLife.84630.1']
  session_description: my first test recording
  session_id: exp1
  session_start_time: 2023-03-08 12:06:36.468983-08:00
  subject: subject pynwb.file.Subject at 0x2239210054728
Fields:
  age: P90D
  description: mouse000000
  sex: M
  species: Mus musculus
  strain: C57BL/6J
  subject_id: 001

  timestamps_reference_time: 2023-03-08 12:06:36.468983-08:00



## Add trial times

Trials are stored in :py:class:`pynwb.epoch.TimeIntervals` object which is
a subclass of :py:class:`pynwb.core.DynamicTable`.
:py:class:`pynwb.core.DynamicTable` objects are used to store tabular metadata
throughout NWB, including trials, electrodes and sorted units. They offer
flexibility for tabular data by allowing required columns, optional columns,
and custom columns which are not defined in the standard.

*missing image*

The ``trials`` :py:class:`pynwb.core.DynamicTable` can be thought of
as a table with this structure:

*missing image*

Trials can be added to the :py:class:`~pynwb.file.NWBFile` using the
methods :py:meth:`~pynwb.file.NWBFile.add_trial_column` and :py:meth:`~pynwb.file.NWBFile.add_trial`
We can add custom, user-defined columns to the trials table to hold data
and metadata specific to this experiment or session.
By default, :py:class:`~pynwb.file.NWBFile` only requires the ``start_time``
and ``end_time`` of the trial. Additional columns can be added using
the :py:meth:`~pynwb.file.NWBFile.add_trial_column` method.

Continue adding to our :py:class:`~pynwb.file.NWBFile` by creating a new
column for the trials table named ``'correct'``, which will be a boolean array.
Once all columns have been added, trial data can be populated using
:py:meth:`~pynwb.file.NWBFile.add_trial`.

AllenSDK first iterates through the rows of the stim dataframe and adds them to the NWBfile using `nwbfile.add_trial()`. Then it iterates through the extra columns and adds them and the associated data (all rows) using `nwbfile.add_trial_column()`.

In [8]:
## Create a dataframe of fake trials ##
n_trials = 20
onsets = np.sort(np.random.uniform(low=0.0, high=100., size=n_trials))
offsets = onsets + 0.5
stimtype = ['electrical'] * n_trials
curr = np.random.choice(['30', '50', '70'], size=n_trials)
valid = np.random.random(size=n_trials) > 0.25

fake_trials_df = pd.DataFrame({
    'start_time': onsets,
    'stop_time': offsets,
    'stimulation_type': stimtype,
    'stimulation_parameter': curr,
    'is_valid': valid,
})

column_desc_dict={
    'stimulation_type': 'type of stimulation delivered',
    'stimulation_parameter': 'specifics of stimulus delivered; if electrical, parameter given is current (\u03bcA)',
    'is_valid': 'is this a valid trial',
}

In [9]:
fake_trials_df.head()

,start_time,stop_time,stimulation_type,stimulation_parameter,is_valid
0,6.037512,6.537512,electrical,70,True
1,9.461760,9.961760,electrical,50,False
2,10.465795,10.965795,electrical,70,True
3,16.613032,17.113032,electrical,70,True
4,18.431073,18.931073,electrical,30,False


In [10]:
## Easier way to do this, using .from_dataframe ##
nwbfile.trials = TimeIntervals.from_dataframe(
    name = "trials", # name must be "trials"
    df = fake_trials_df
)

In [11]:
nwbfile.trials

trials pynwb.epoch.TimeIntervals at 0x2239210040456
Fields:
  colnames: ['start_time' 'stop_time' 'stimulation_type' 'stimulation_parameter'
 'is_valid']
  columns: (
    start_time <class 'hdmf.common.table.VectorData'>,
    stop_time <class 'hdmf.common.table.VectorData'>,
    stimulation_type <class 'hdmf.common.table.VectorData'>,
    stimulation_parameter <class 'hdmf.common.table.VectorData'>,
    is_valid <class 'hdmf.common.table.VectorData'>
  )
  id: id <class 'hdmf.common.table.ElementIdentifiers'>

In [12]:
nwbfile.trials.to_dataframe()

,start_time,stop_time,stimulation_type,stimulation_parameter,is_valid
id,,,,,
0,6.037512,6.537512,electrical,70,True
1,9.461760,9.961760,electrical,50,False
2,10.465795,10.965795,electrical,70,True
3,16.613032,17.113032,electrical,70,True
4,18.431073,18.931073,electrical,30,False
5,40.697099,41.197099,electrical,50,False
6,44.461347,44.961347,electrical,50,True
7,44.538586,45.038586,electrical,30,True
8,44.899864,45.399864,electrical,50,True


## Add running speed

:py:class:`~pynwb.base.TimeSeries` is a common base class for measurements sampled over time,
and provides fields for ``data`` and ``timestamps`` (regularly or irregularly sampled).
You will also need to supply the ``name`` and ``unit`` of measurement
([SI unit](https://en.wikipedia.org/wiki/International_System_of_Units)).

*missing image*

For instance, we can store a :py:class:`~pynwb.base.TimeSeries` data where recording started
``0.0`` seconds after ``start_time`` and sampled every second:

The AllenSDK-ecephys uses the pynwb.base.TimeSeries class to set the running speed, but they do not wrap it as a BehaviorTimeSeries.

In [13]:
rundata = np.linspace(0, 20, 10).astype(float)
runtimestamps = np.linspace(0, 11, 10).astype(float)

speed_with_timestamps = TimeSeries(
    name = "running_speed",
    data = rundata,
    unit = "cm/s",
#     starting_time = float(runtimestamps[0]), # must be a float, not int
#     rate = 100.0,
    timestamps = runtimestamps,
    description = "running speed data, computed from wheel angular velocity"
)
## CANNOT specify rate AND timestamps...

Make the TimeSeries into a BehavioralTimeSeries...which is for storing continuous behavior data, such as speed.

In [14]:
behavioral_time_series = BehavioralTimeSeries(
    time_series = speed_with_timestamps,
    name = "BehavioralTimeSeries", # can rename, it dictates the key that is used to call up this info
)

behavior_module = nwbfile.create_processing_module(
    name="behavior", description="processed behavioral data"
)
behavior_module.add(behavioral_time_series)

BehavioralTimeSeries pynwb.behavior.BehavioralTimeSeries at 0x2239210473992
Fields:
  time_series: {
    running_speed <class 'pynwb.base.TimeSeries'>
  }

In [15]:
print(nwbfile.processing["behavior"].children)

(BehavioralTimeSeries pynwb.behavior.BehavioralTimeSeries at 0x2239210473992
Fields:
  time_series: {
    running_speed <class 'pynwb.base.TimeSeries'>
  }
,)


In [16]:
print(nwbfile.processing["behavior"]["BehavioralTimeSeries"]["running_speed"])

running_speed pynwb.base.TimeSeries at 0x2239210472712
Fields:
  comments: no comments
  conversion: 1.0
  data: [ 0.          2.22222222  4.44444444  6.66666667  8.88888889 11.11111111
 13.33333333 15.55555556 17.77777778 20.        ]
  description: running speed data, computed from wheel angular velocity
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: [ 0.          1.22222222  2.44444444  3.66666667  4.88888889  6.11111111
  7.33333333  8.55555556  9.77777778 11.        ]
  timestamps_unit: seconds
  unit: cm/s



## Add EEG array as an electrodes table

In order to store extracellular electrophysiology data, you first must create an electrodes table
describing the electrodes that generated this data. Extracellular electrodes are stored in an
``"electrodes"`` table, which is a :py:class:`~hdmf.common.table.DynamicTable`.
<br>
Before creating an :py:class:`~pynwb.ecephys.ElectrodeGroup`, you need to provide some information about the
device that was used to record from the electrode. This is done by creating a :py:class:`~pynwb.device.Device`
object using the instance method :py:meth:`~pynwb.file.NWBFile.create_device`.

In [18]:
eegdevice = nwbfile.create_device(
    name = "EEG array",
    description = "H32 Mouse EEG (30-ch)", # can use Neuronexus description
    manufacturer = "Neuronexus",
)

Once you have created the :py:class:`~pynwb.device.Device`, you can create an
:py:class:`~pynwb.ecephys.ElectrodeGroup`. Then you can add electrodes one-at-a-time with
:py:meth:`~pynwb.file.NWBFile.add_electrode`. :py:meth:`~pynwb.file.NWBFile.add_electrode` has two required arguments,
``group``, which takes an :py:class:`~pynwb.ecephys.ElectrodeGroup`, and ``location``, which takes a string. It also
has a number of optional metadata fields for electrode features (e.g, ``x``, ``y``, ``z``, ``imp``,
and ``filtering``). Since this table is a :py:class:`~hdmf.common.table.DynamicTable`, we can add
additional user-specified metadata fields as well. We will be adding a ``"label"`` column to the table. Use the
following code to add electrodes for an array with 4 shanks and 3 channels per shank.

### Add 30 electrodes in 1 group

In [19]:
EEG_elec_group = nwbfile.create_electrode_group(
    name = "EEG array",
    description = "30-ch array on the skull surface", # could include ref and gnd location?
    device = eegdevice,
    location = "skull surface, both hemispheres", # is both hemispheres necessary?
)

for chi in range(len(EEGexp.EEG_channel_coordinates)):
    nwbfile.add_electrode(
        group = EEG_elec_group,
        location = "brain area",
        reference = "stainless steel skull screw over L cerebellum", # can include string detailing ref electrode
    )

TypeError: MultiContainerInterface.__make_create.<locals>._func: unrecognized argument: 'sampling_rate'

In [19]:
nwbfile.electrodes.to_dataframe()

,location,group,group_name,reference
id,,,,
0,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
1,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
2,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
3,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
4,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
5,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
6,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
7,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
8,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum


**All electrodes will be added to the same DynamicTable, EEG and all NPXs.** Will need to keep track, so the DynamicTableRegion can be assigned accordingly.

### Add some data

#### First, create a DynamicTableRegion

Raw voltage traces and local-field potential (LFP) data are stored in :py:class:`~pynwb.ecephys.ElectricalSeries`
objects. :py:class:`~pynwb.ecephys.ElectricalSeries` is a subclass of :py:class:`~pynwb.base.TimeSeries`
specialized for voltage data. To create the :py:class:`~pynwb.ecephys.ElectricalSeries` objects, we need to
reference a set of rows in the ``"electrodes"`` table to indicate which electrodes were recorded. We will do this
by creating a :py:class:`~pynwb.core.DynamicTableRegion`, which is a type of link that allows you to reference
:py:meth:`~pynwb.file.NWBFile.create_electrode_table_region` is a convenience function that creates a
:py:class:`~pynwb.core.DynamicTableRegion` which references the ``"electrodes"`` table.

In [20]:
EEG_table_region = nwbfile.create_electrode_table_region(
    region=list(range(len(EEGexp.EEG_channel_coordinates))),  # reference rows associated with EEG array
    description="EEG electrodes",
)

#### Now add some nonsense data as an ElectricalSeries

Now create an :py:class:`~pynwb.ecephys.ElectricalSeries` object to store raw data collected
during the experiment, passing in this ``"EEG_table_region"`` :py:class:`~pynwb.core.DynamicTableRegion`
reference to all rows of the electrodes table.

*missing image*

**Can the name be different/more discriptive?**

In [21]:
raw_EEG_data = np.random.randn(50, len(EEGexp.EEG_channel_coordinates))
EEG_electrical_series = ElectricalSeries(
    name="ElectricalSeries", # this is the key used to call up the data within the NWBfile, BP: ElectricalSeriesFromEEG, etc.
    data=raw_EEG_data,
    electrodes=EEG_table_region,
    starting_time=0.0,  # timestamp of the first sample in seconds relative to the session start time
    rate=2500.0,  # in Hz
#     timestamps= , # can provide timestamps
)

NWB organizes data into different groups depending on the type of data. Groups can be thought of
as folders within the file. Here are some of the groups within an :py:class:`~pynwb.file.NWBFile` and the types of
data they are intended to store:

* **acquisition**: raw, acquired data that should never change
* **processing**: processed data, typically the results of preprocessing algorithms and could change
* **analysis**: results of data analysis
* **stimuli**: stimuli used in the experiment (e.g., images, videos, light pulses)

Since this :py:class:`~pynwb.ecephys.ElectricalSeries` represents raw data from the data acquisition system,
we will add it to the acquisition group of the :py:class:`~pynwb.file.NWBFile`.

In [22]:
nwbfile.add_acquisition(EEG_electrical_series)

In [23]:
print(nwbfile)

root pynwb.file.NWBFile at 0x2387111115912
Fields:
  acquisition: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    EEG array <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    EEG array <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: test making NWB file
  file_create_date: [datetime.datetime(2023, 2, 17, 14, 34, 49, 372324, tzinfo=tzlocal())]
  identifier: 000000_day1
  institution: Allen Institute
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>
  }
  related_publications: ['https://doi.org/10.7554/eLife.84630.1']
  session_description: my first test recording
  session_id: exp1
  session_start_time: 2023-02-17 14:34:49.371325-08:00
  subject: subject pynwb.file.Subject at 0x2387111267656
Fields:
  age: P90D
  description: mouse000000
  sex: M
  species: Mus musculus
  strain: C57BL/6J
  subject_id: 001

  timestamps_reference_time: 202

## Add NPX device

In [24]:
numNPXchs = 20

In [25]:
NPXFdevice = nwbfile.create_device(
    name = "ProbeF",
    description = "Neuropixels 1.0 probe", # say 1.0 or 3a
    manufacturer = "imec",
)
## AllenSDK includes "probe_id" and "sampling_rate"

In [26]:
NPXF_elec_group = nwbfile.create_electrode_group(
    name = "ProbeF",
    description = "Neuropixels probe F",
    device = NPXFdevice,
    location = "left MOs", # give surface loc?
)
## AllenSDK sets location = "See electrode locations" since those will be more accurate anyway

for chi in range(numNPXchs):
    nwbfile.add_electrode(
        group = NPXF_elec_group,
        location = "deep area",
        reference = "ProbeF tip reference electrode", # must include ref here if we include it for EEG
    )

In [27]:
nwbfile.electrodes.to_dataframe()

,location,group,group_name,reference
id,,,,
0,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
1,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
2,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
3,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
4,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
5,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
6,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
7,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum
8,brain area,EEG array pynwb.ecephys.ElectrodeGroup at 0x23...,EEG array,stainless steel skull screw over L cerebellum


### Now add some nonsense data as LFP

Create an :py:class:`~pynwb.ecephys.ElectricalSeries` object to store LFP data collected during the experiment,
again passing in the :py:class:`~pynwb.core.DynamicTableRegion` reference to the ProbeF rows of the ``"electrodes"`` table.

In [28]:
Felec_table_region = nwbfile.create_electrode_table_region(
    region=list(range(len(EEGexp.EEG_channel_coordinates), len(EEGexp.EEG_channel_coordinates)+numNPXchs)),  # rows associated w/ F
    description="ProbeF electrodes",
)

**Can the name be different/more discriptive?**

In [29]:
lfp_data = np.random.randn(50, numNPXchs)
lfp_electrical_series = ElectricalSeries(
    name="ElectricalSeries",
    data=lfp_data,
    electrodes=Felec_table_region,
    starting_time=0.0,
    rate=2500.0,
)

To help data analysis and visualization tools know that this :py:class:`~pynwb.ecephys.ElectricalSeries` object
represents LFP data, store the :py:class:`~pynwb.ecephys.ElectricalSeries` object inside of an
:py:class:`~pynwb.ecephys.LFP` object. This is analogous to how we can store the
:py:class:`~pynwb.behavior.SpatialSeries` object inside of a :py:class:`~pynwb.behavior.Position` object.

*missing image*

In [30]:
lfp = LFP(electrical_series=lfp_electrical_series)

Unlike the raw data, which we put into the acquisition group of the :py:class:`~pynwb.file.NWBFile`,
LFP data is typically considered processed data because the raw data was filtered and downsampled to generate the LFP.

Create a processing module named ``"ecephys"`` and add the :py:class:`~pynwb.ecephys.LFP` object to it.
This is analogous to how we can store the :py:class:`~pynwb.behavior.Position` object in a processing module
created with the :py:class:`~pynwb.file.NWBFile.create_processing_module` method.

In [31]:
ecephys_module = nwbfile.create_processing_module(
    name="ecephys", description="processed extracellular electrophysiology data"
)
ecephys_module.add(lfp)

LFP pynwb.ecephys.LFP at 0x2387111906568
Fields:
  electrical_series: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }

In [32]:
print(nwbfile)

root pynwb.file.NWBFile at 0x2387111115912
Fields:
  acquisition: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    EEG array <class 'pynwb.device.Device'>,
    ProbeF <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    EEG array <class 'pynwb.ecephys.ElectrodeGroup'>,
    ProbeF <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: test making NWB file
  file_create_date: [datetime.datetime(2023, 2, 17, 14, 34, 49, 372324, tzinfo=tzlocal())]
  identifier: 000000_day1
  institution: Allen Institute
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>,
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  related_publications: ['https://doi.org/10.7554/eLife.84630.1']
  session_description: my first test recording
  session_id: exp1
  session_start_time: 2023-02-17 14:34:49.371325-08:00
  subject: subject pynwb.file.Subject at 0x2387111267656
Fields

## Add units with spiketimes

Spike times are stored in the :py:class:`~pynwb.misc.Units` table, which is a subclass of
:py:class:`~hdmf.common.table.DynamicTable`. Adding columns to the :py:class:`~pynwb.misc.Units` table is analogous
to how we can add columns to the ``"electrodes"`` and ``"trials"`` tables.

We will generate some random spike data and populate the :py:meth:`~pynwb.misc.Units` table using the
:py:class:`~pynwb.file.NWBFile.add_unit` method. Then we can display the :py:class:`~pynwb.misc.Units` table as a
pandas :py:class:`~pandas.DataFrame`.

Use the following class method, including parameters:
<br>`add_unit(spike_times=None, obs_intervals=None, electrodes=None, electrode_group=None, waveform_mean=None, waveform_sd=None, waveforms=None, id=None)`
<br>Can include other parameters using `add_unit_column` to name new columns.

In [33]:
## Fake units and firing parameters ##
n_units = 15
poisson_lambda = 20
firing_rate = 20
unit_elec = np.random.choice(range(len(EEGexp.EEG_channel_coordinates), len(EEGexp.EEG_channel_coordinates)+numNPXchs), size=n_units)

for unitn in range(n_units):
    n_spikes = np.random.poisson(lam=poisson_lambda)
    spike_times = np.round(
        np.cumsum(np.random.exponential(1 / firing_rate, n_spikes)), 5
    )
    nwbfile.add_unit(
        spike_times=spike_times,
        electrodes=[unit_elec[unitn]], # the electrodes that the unit came from, must be array-like, this appears to ref the electrodes table
        electrode_group=NPXF_elec_group, # the electrode group that each unit came from
        waveform_mean=[1.0, 2.0, 3.0, 4.0, 5.0], # shape is (time,) or (time, electrodes), not required
#         id=2+unitn, # must be int! becomes the index
    )

nwbfile.units.to_dataframe()

,spike_times,electrodes,electrode_group,waveform_mean
id,,,,
0,"[0.00115, 0.17605, 0.3015, 0.30287, 0.42288, 0...",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"
1,"[0.05509, 0.05591, 0.08798, 0.1859, 0.24202, 0...",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"
2,"[0.01907, 0.10874, 0.13677, 0.14427, 0.17662, ...",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"
3,"[0.04037, 0.15405, 0.21998, 0.22739, 0.269, 0....",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"
4,"[0.02125, 0.02335, 0.05755, 0.05933, 0.08546, ...",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"
5,"[0.1187, 0.24906, 0.26489, 0.27419, 0.33388, 0...",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"
6,"[0.05824, 0.10028, 0.17995, 0.21056, 0.2193, 0...",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"
7,"[0.05904, 0.07532, 0.13762, 0.1838, 0.44017, 0...",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"
8,"[0.06018, 0.11766, 0.19675, 0.19832, 0.20033, ...",location ...,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,"[1.0, 2.0, 3.0, 4.0, 5.0]"


In [34]:
nwbfile.units[3, 'electrodes']

,location,group,group_name,reference
id,,,,
34,deep area,ProbeF pynwb.ecephys.ElectrodeGroup at 0x23871...,ProbeF,ProbeF tip reference electrode


AllenSDK-ecephys creates the Units table using `pynwb.misc.Units.from_dataframe()`. To use this we would need to make a giant dataframe containing units from all probes first.

See https://github.com/AllenInstitute/AllenSDK/blob/1caa779b517eeb2428282919d0fb8a65e7217791/allensdk/brain_observatory/ecephys/probes.py#L211 and https://github.com/AllenInstitute/AllenSDK/blob/1caa779b517eeb2428282919d0fb8a65e7217791/allensdk/brain_observatory/ecephys/nwb_util.py#L23

**Creating it this way does not allow for an easy link to the electrodes table, like the above way does. So this may not be ideal.**

#### Designating electrophysiology data

As mentioned above, :py:class:`~pynwb.ecephys.ElectricalSeries` objects
are meant for storing specific types of extracellular recordings. In addition to this
:py:class:`~pynwb.base.TimeSeries` class, NWB provides some `modules_overview`
for designating the type of data you are storing. We will briefly discuss them here, and refer the reader to
:py:mod:`API documentation <pynwb.ecephys>` and `basics` for more details on
using these objects.

For storing spike data, there are two options. Which one you choose depends on what data you have available.
If you need to store the complete, continuous raw voltage traces, you should store your the traces with
:py:class:`~pynwb.ecephys.ElectricalSeries` objects as `acquisition <basic_timeseries>` data, and use
the :py:class:`~pynwb.ecephys.EventDetection` class for identifying the spike events in your raw traces.
If you do not want to store the raw voltage traces and only the waveform 'snippets' surrounding spike events,
you should use the :py:class:`~pynwb.ecephys.EventWaveform` class, which can store one or more
:py:class:`~pynwb.ecephys.SpikeEventSeries` objects.

The results of spike sorting (or clustering) should be stored in the top-level :py:class:`~pynwb.misc.Units` table.
Note that it is not required to store spike waveforms in order to store spike events or waveforms--if you only
want to store the spike times of clustered units you can use only the Units table.

For local field potential data, there are two options. Again, which one you choose depends on what data you
have available. With both options, you should store your traces with :py:class:`~pynwb.ecephys.ElectricalSeries`
objects. If you are storing unfiltered local field potential data, you should store
the :py:class:`~pynwb.ecephys.ElectricalSeries` objects in :py:class:`~pynwb.ecephys.LFP` data interface object(s).
If you have filtered LFP data, you should store the :py:class:`~pynwb.ecephys.ElectricalSeries` objects  in
:py:class:`~pynwb.ecephys.FilteredEphys` data interface object(s).


In [35]:
nwbfile.acquisition.keys()

dict_keys(['ElectricalSeries'])

## Write the NWBfile

Once you have finished adding all of your data to the :py:class:`~pynwb.file.NWBFile`,
write the file with :py:class:`~pynwb.NWBHDF5IO`.

In [36]:
my_test_file = os.path.join(NWB_output_dir, r'my_first_test_nwbfile.nwb')

In [37]:
start = time.time()
with NWBHDF5IO(my_test_file, "w") as io:
    io.write(nwbfile, cache_spec=True)
end = time.time()
print('Time to write NWB file: {:.2f} s'.format(end-start))

Time to write NWB file: 1.16 s


Not sure why I am seeing the warning here...actually I think it was the running_timestamps, which I had set as an array of ints. Should be fixed now.
<br>NWBfile was written pretty quickly and this test file is ~250 KB.